---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# READ DATA

In [31]:
import pandas as pd
df = pd.read_csv(
    "https://www.dropbox.com/scl/fi/x61ewvoj9spslqblqweke/stocks_bonds_gold.csv?rlkey=ssmouraepasrj95b4rxj26bpk&dl=1",
    index_col="Year"
)

# COMPUTE HISTORICAL PARAMETERS

In [32]:
mn = df.mean()
sd = df.std()
cov = df.cov()

num_assets = df.shape[1]

# EXAMPLE DATA

In [33]:
# risk-free rate
rf = 0.02

# or savings rate and borrowing rate
rs = 0.02
rb = 0.05

# risk aversion
raver = 6

# INSTALL LIBRARIES

In [34]:
!pip install cvxopt


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# OPTIMAL PORTFOLIO 1

Use this if the borrowing and savings rate are the same and short sales are allowed.

In [35]:
import numpy as np

optimum = np.linalg.solve(raver*cov, mn-rf)
optimum_mn = rf + optimum @ (mn-rf)
optimum_sd = np.sqrt(optimum @ cov @ optimum)

print(f"optimal savings is {max(0, 1-np.sum(optimum)): 0.3f}")
print(f"optimal borrowing is {max(0, np.sum(optimum)-1): 0.3f}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal standard deviation is {optimum_sd: 0.3f}")
print("optimal portfolio is")
pd.Series(optimum, index=df.columns)

optimal savings is  0.000
optimal borrowing is  0.971
optimal expected return is  0.162
optimal standard deviation is  0.154
optimal portfolio is


Stocks        0.272661
Treasuries   -0.000869
Corporates    1.421607
Gold          0.277699
dtype: float64

# OPTIMAL PORTFOLIO 2

Use this if the borrowing rate is greater than the savings rate and short sales are allowed.    

In [36]:
from cvxopt import matrix
from cvxopt.solvers import qp

# to define variance penalty
P = np.zeros((num_assets+2, num_assets+2))
P[2:,2:] = raver*cov

# to define (minus) expected return 
q = np.concatenate(
    (
        np.array([-rs, rb]),
        -mn
    )
)

# to impose -xs <= 0 and -xb <= 0
G = np.zeros((2, num_assets+2))
G[0, 0] = G[1, 1] = -1
h = np.zeros(2)

# to impose xs - xb + w1 + w2 + w3 = 1
A = np.ones(num_assets+2)
A[1] = -1
b = [1.]

# create cvxopt matrix objects
P = matrix(P, (num_assets+2, num_assets+2))
q = matrix(q, (num_assets+2, 1))
G = matrix(G, (2, num_assets+2))
h = matrix(h, (2, 1))
A = matrix(A, (1, num_assets+2))
b = matrix(b, (1, 1))

# compute optimum by quadratic programming
solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
optimum = np.array(solution["x"])
xs, xb = optimum[0].item(), optimum[1].item()
optimum = optimum[2:].reshape(-1, )

# expected return and risk
optimum_mn = xs*rs - xb*rb + optimum @ mn
optimum_sd = np.sqrt(optimum @ cov @ optimum)

print(f"optimal savings is {xs: 0.3f}")
print(f"optimal borrowing is {xb: 0.3f}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal standard deviation is {optimum_sd: 0.3f}")
print("optimal portfolio is")
pd.Series(optimum, index=df.columns)

     pcost       dcost       gap    pres   dres
 0: -7.6461e-02 -4.5512e-02  2e+00  1e+00  1e+00
 1: -4.7458e-02 -9.2833e-02  5e-02  1e-02  1e-02
 2: -7.1934e-02 -7.6135e-02  4e-03  3e-16  1e-17
 3: -7.5450e-02 -7.5715e-02  3e-04  1e-17  1e-17
 4: -7.5545e-02 -7.5575e-02  3e-05  3e-17  2e-17
 5: -7.5549e-02 -7.5551e-02  2e-06  7e-18  1e-17
 6: -7.5549e-02 -7.5549e-02  2e-08  1e-16  8e-18
Optimal solution found.
optimal savings is  0.000
optimal borrowing is  0.048
optimal expected return is  0.101
optimal standard deviation is  0.092
optimal portfolio is


Stocks        0.207019
Treasuries   -0.245202
Corporates    0.913609
Gold          0.172262
dtype: float64

# OPTIMAL PORTFOLIO 3

Use this if short sales are not allowed.  If the borrowing and savings rate are the same (=rf) then add the line:

    rb = rs = rf

In [37]:
from cvxopt import matrix
from cvxopt.solvers import qp

# to define variance penalty
P = np.zeros((num_assets+2, num_assets+2))
P[2:,2:] = raver*cov

# to define (minus) expected return 
q = np.concatenate(
    (
        np.array([-rs, rb]),
        -mn
    )
)

# to impose -xs <= 0 and -xb <= 0 and -w <= 0
G = -np.identity(num_assets+2)
h = np.zeros(num_assets+2)

# to impose xs - xb + w1 + w2 + w3 = 1
A = np.ones(num_assets+2)
A[1] = -1
b = [1.]

# create cvxopt matrix objects
P = matrix(P, (num_assets+2, num_assets+2))
q = matrix(q, (num_assets+2, 1))
G = matrix(G, (num_assets+2, num_assets+2))
h = matrix(h, (num_assets+2, 1))
A = matrix(A, (1, num_assets+2))
b = matrix(b, (1, 1))

# compute optimum by quadratic programming
solution = qp(P=P, q=q, G=G, h=h, A=A, b=b)
optimum = np.array(solution["x"])
xs, xb = optimum[0].item(), optimum[1].item()
optimum = optimum[2:].reshape(-1, )

# expected return and risk
optimum_mn = xs*rs - xb*rb + optimum @ mn
optimum_sd = np.sqrt(optimum @ cov @ optimum)

print(f"optimal savings is {xs: 0.3f}")
print(f"optimal borrowing is {xb: 0.3f}")
print(f"optimal expected return is {optimum_mn: 0.3f}")
print(f"optimal std dev is {optimum_sd: 0.3f}")
print("optimal portfolio is")
pd.Series(optimum, index=df.columns)

     pcost       dcost       gap    pres   dres
 0: -6.4726e-02 -6.9624e-01  8e+00  3e+00  3e+00
 1: -3.2927e-02 -1.3307e+00  2e+00  6e-01  6e-01
 2: -1.3557e-02 -1.5587e-01  1e-01  6e-15  2e-16
 3: -6.9607e-02 -8.5846e-02  2e-02  4e-16  4e-17
 4: -7.4077e-02 -7.5739e-02  2e-03  2e-16  1e-17
 5: -7.4641e-02 -7.4699e-02  6e-05  1e-16  1e-17
 6: -7.4655e-02 -7.4656e-02  8e-07  6e-17  1e-17
 7: -7.4656e-02 -7.4656e-02  8e-09  1e-16  2e-17
Optimal solution found.
optimal savings is  0.000
optimal borrowing is  0.107
optimal expected return is  0.099
optimal std dev is  0.091
optimal portfolio is


Stocks        2.501792e-01
Treasuries    1.713079e-07
Corporates    6.832103e-01
Gold          1.736737e-01
dtype: float64